In [1]:
import os
import pandas as pd
import numpy as np
from pandas import read_csv
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import ADASYN

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
df=pd.read_excel('/content/gdrive/MyDrive/Btech_Project/CombinedDataset/Equ_Myl_Ecl.xlsx')
df

,classname,numberOfBugsFoundUntil:,numberOfNonTrivialBugsFoundUntil:,numberOfMajorBugsFoundUntil:,numberOfCriticalBugsFoundUntil:,numberOfHighPriorityBugsFoundUntil:,Nondefective,nonTrivialBugs,majorBugs,criticalBugs,...,numberOfAttributesInherited,numberOfLinesOfCode,numberOfMethods,numberOfMethodsInherited,numberOfPrivateAttributes,numberOfPrivateMethods,numberOfPublicAttributes,numberOfPublicMethods,rfc,wmc
0,org::eclipse::mylyn::internal::trac::core::uti...,6,4,0,0,1,1,0,0,0,...,0,42,6,9,0,1,0,5,18,8
1,org::eclipse::mylyn::internal::sandbox::bridge...,12,6,0,0,10,1,0,0,0,...,0,60,9,0,6,0,0,8,29,14
2,org::eclipse::mylyn::internal::jira::core::mod...,6,3,0,0,5,1,0,0,0,...,0,13,4,19,3,0,0,3,5,4
3,org::eclipse::mylyn::internal::wikitext::texti...,3,3,0,0,0,1,0,0,0,...,0,12,4,22,0,0,0,1,6,3
4,org::eclipse::mylyn::internal::java::ui::searc...,3,0,0,0,2,1,0,0,0,...,8,12,4,47,0,0,0,2,4,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3178,org::eclipse::jdt::internal::compiler::codegen...,10,9,2,2,2,1,0,0,0,...,0,3,2,8,0,0,2,2,2,1
3179,org::eclipse::jdt::internal::compiler::ast::Ja...,23,21,3,1,1,1,0,0,0,...,386,48,6,95,0,1,3,3,27,14
3180,org::eclipse::jdt::internal::core::search::mat...,21,18,1,0,0,0,0,0,0,...,52,306,18,27,2,0,6,6,94,88
3181,org::eclipse::jdt::core::dom::RecoveredVariabl...,4,2,0,0,0,1,0,0,0,...,6,87,20,30,2,0,0,19,47,29


In [4]:
print("defective ",len(df[df['Nondefective'] ==0]))

defective  580


In [5]:
unlabelled_df = df.loc[(df['Nondefective']==0)&(df[' highPriorityBugs ']==0)&(df[' criticalBugs ']==0)&(df[' majorBugs ']==0)&(df[' nonTrivialBugs ']==0)]

In [6]:
print(unlabelled_df.shape)

(377, 43)


In [7]:
labelled_df = df.loc[~((df['Nondefective']==0)&(df[' highPriorityBugs ']==0)&(df[' criticalBugs ']==0)&(df[' majorBugs ']==0)&(df[' nonTrivialBugs ']==0))]

In [8]:
print(labelled_df.shape)

(2806, 43)


In [9]:
df.columns

Index(['classname ', ' numberOfBugsFoundUntil: ',
       ' numberOfNonTrivialBugsFoundUntil: ', ' numberOfMajorBugsFoundUntil: ',
       ' numberOfCriticalBugsFoundUntil: ',
       ' numberOfHighPriorityBugsFoundUntil: ', 'Nondefective',
       ' nonTrivialBugs ', ' majorBugs ', ' criticalBugs ',
       ' highPriorityBugs ', ' numberOfVersionsUntil: ',
       ' numberOfFixesUntil: ', ' numberOfRefactoringsUntil: ',
       ' numberOfAuthorsUntil: ', ' linesAddedUntil: ',
       ' maxLinesAddedUntil: ', ' avgLinesAddedUntil: ',
       ' linesRemovedUntil: ', ' maxLinesRemovedUntil: ',
       ' avgLinesRemovedUntil: ', ' codeChurnUntil: ', ' maxCodeChurnUntil: ',
       ' avgCodeChurnUntil: ', ' ageWithRespectTo: ',
       ' weightedAgeWithRespectTo: ', ' cbo ', ' dit ', ' fanIn ', ' fanOut ',
       ' lcom ', ' noc ', ' numberOfAttributes ',
       ' numberOfAttributesInherited ', ' numberOfLinesOfCode ',
       ' numberOfMethods ', ' numberOfMethodsInherited ',
       ' numberOfPrivateA

# Generating **X,y**

In [10]:
data =labelled_df.values
idx_OUT_columns = [10,9,8,7,6]
idx_IN_columns = [i for i in range(np.shape(data)[1]) if i not in idx_OUT_columns]
x = data[:,idx_IN_columns]
x = x[:,1:]
y = data[:,idx_OUT_columns]
y=y.astype('int')
print("x : ",x)
print("y : ",y)

x :  [[6 4 0 ... 5 18 8]
 [12 6 0 ... 8 29 14]
 [6 3 0 ... 3 5 4]
 ...
 [10 9 2 ... 2 2 1]
 [23 21 3 ... 3 27 14]
 [4 2 0 ... 19 47 29]]
y :  [[0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 ...
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]]


In [11]:
data =unlabelled_df.values
idx_OUT_columns = [10,9,8,7,6]
idx_IN_columns = [i for i in range(np.shape(data)[1]) if i not in idx_OUT_columns]
X_unlabelled = data[:,idx_IN_columns]
X_unlabelled = X_unlabelled[:,1:]

In [12]:
oversample = ADASYN(n_neighbors=1)
X, Y = oversample.fit_resample(x, y)

# **CLASSESS**

In [13]:
def counting(y):
  count=[0,0,0,0,0]
  for i in range(len(y)):
    k=0
    for j in range(5):
      if(y[i][j]==1):
        count[j]+=1
  print("no of nonTrivialBugs ",count[3])
  print("no of majorBugs ",count[2])
  print("no of criticalBugs ",count[1])
  print("no of highPriorityBugs ",count[0])
  print("no of Nondefective modules ",count[4])
  return count;

In [14]:
counting(Y)

no of nonTrivialBugs  2575
no of majorBugs  2602
no of criticalBugs  2600
no of highPriorityBugs  2587
no of Nondefective modules  2603


[2587, 2600, 2602, 2575, 2603]

# SELF-**TRAINING**

In [15]:
# Initiate iteration counter

iterations = 0

# Containers to hold f1_scores and # of pseudo-labels
train_f1s = []
test_f1s = []
pseudo_labels = []

# Assign value to initiate while loop
high_prob = [1] 

Y=np.argmax(Y,axis=1)

cnt=0
while len(X_unlabelled) > 0:
        
    # Fit classifier and make train/test predictions
    clf =  DecisionTreeClassifier()
    clf.fit(X, Y)
    y_hat_train = clf.predict(X)

    # Calculate and print iteration # and f1 scores, and store f1 scores
    train_f1 = f1_score(Y, y_hat_train,average='micro')
    print(f"Iteration {iterations}") 
    print(f"Train f1: {train_f1}")
    train_f1s.append(train_f1)
   
    # Generate predictions and probabilities for unlabeled data
    print(f"Now predicting labels for unlabeled data...")

    preds = clf.predict(X_unlabelled)
    pred_probs = clf.predict_proba(X_unlabelled)

    prob_0 = [row[0] for row in pred_probs]
    prob_1 = [row[1] for row in pred_probs]
    prob_2 = [row[2] for row in pred_probs]
    prob_3 = [row[3] for row in pred_probs]
    prob_4 = [row[4] for row in pred_probs]

    # Store predictions and probabilities in dataframe
    df_pred_prob = pd.DataFrame([])
   
    df_pred_prob['preds'] = preds
    df_pred_prob['prob_0'] = prob_0
    df_pred_prob['prob_1'] = prob_1
    df_pred_prob['prob_2'] = prob_2
    df_pred_prob['prob_3'] = prob_3
    df_pred_prob['prob_4'] = prob_4 
    df_pred_prob.index = [i for i in range(len(X_unlabelled))]
    k=0.99
    # Separate predictions with > 99% probability
    high_prob = pd.concat([df_pred_prob.loc[df_pred_prob['prob_0'] > k],
                           df_pred_prob.loc[df_pred_prob['prob_1'] > k],df_pred_prob.loc[df_pred_prob['prob_2'] > k],
                           df_pred_prob.loc[df_pred_prob['prob_3'] > k],df_pred_prob.loc[df_pred_prob['prob_4'] > k]],
                          axis=0)
    
    print(f"{len(high_prob)} high-probability predictions added to training data.")
    
    pseudo_labels.append(len(high_prob))

    # Add pseudo-labeled data to training data
    if(high_prob.empty==False):
      X = np.vstack((X,X_unlabelled[high_prob.index]))
      y_pred  = np.array(high_prob.preds)
      Y = np.hstack((Y,y_pred))
      
      # Drop pseudo-labeled instances from unlabeled data
      idx_predict = [i for i in range(np.shape(X_unlabelled)[0]) if i not in high_prob.index]
      X_unlabelled = X_unlabelled[idx_predict]
      print(f"{len(X_unlabelled)} unlabeled instances remaining.\n")
    else :
      break;
    # Update iteration counter
    iterations += 1

Iteration 0
Train f1: 1.0
Now predicting labels for unlabeled data...
377 high-probability predictions added to training data.
0 unlabeled instances remaining.



## **Performance Measures**

In [16]:
def Function(clf):
  clf.fit(X, Y)
  y_hat_train = clf.predict(X)
  train_f1 = f1_score(Y, y_hat_train,average='micro')
  print(f"Train f1: {train_f1}")
  df=pd.read_excel('/content/gdrive/MyDrive/Btech_Project/CombinedDataset/pde.xlsx')
  labelled_df = df.loc[~((df['Nondefective']==0)&(df[' highPriorityBugs ']==0)&(df[' criticalBugs ']==0)&(df[' majorBugs ']==0)&(df[' nonTrivialBugs ']==0))]
  data =labelled_df.values
  idx_OUT_columns = [10,9,8,7,6]
  idx_IN_columns = [i for i in range(np.shape(data)[1]) if i not in idx_OUT_columns]
  x_test = data[:,idx_IN_columns]
  x_test = x_test[:,1:]
  y_test = data[:,idx_OUT_columns]
  y_test=y_test.astype('int')
  y_test=np.argmax(y_test,axis=1)
  yhat_test= clf.predict(x_test)
  yhat_pred_probs = clf.predict_proba(x_test)
  print("test f1score : ",f1_score(y_test, yhat_test,average='micro'))
  CM=confusion_matrix(y_test, yhat_test,labels=[0,1,2,3,4])

  print(CM)

  Risk_HP=((0.1*CM[0][1])+(0.2*CM[0][2])+(0.3*CM[0][3])+(0.4*CM[0][4]))/(CM[0][0]+CM[0][1]+CM[0][2]+CM[0][3]+CM[0][4])
  Risk_C=((0.1*CM[1][2])+(0.2*CM[1][3])+(0.3*CM[1][4]))/(CM[1][0]+CM[1][1]+CM[1][2]+CM[1][3]+CM[1][4])
  Risk_M=((0.1*CM[2][3])+(0.2*CM[2][4]))/(CM[2][0]+CM[2][1]+CM[2][2]+CM[2][3]+CM[2][4])
  Risk_NT=(0.1*CM[3][4])/(CM[3][0]+CM[3][1]+CM[3][2]+CM[3][3]+CM[4][4])
  print("Risk Factor for High Priority",Risk_HP)
  print("Risk Factor for Critical",Risk_C)
  print("Risk Factor for Major",Risk_M)
  print("Risk Factor for Non-Trivial",Risk_NT)
  print('PTN:',CM[4][4],'/',len(x_test),'=',CM[4][4]/len(x_test))
  total_loc=0
  tn_loc=0
  for i in range(len(x_test)):
    total_loc=total_loc+x_test[i][28]
    if((yhat_test[i]==4)and(y_test[i]==4)):
      tn_loc=tn_loc+x_test[i][28]
  print('Saved Budget:',tn_loc)
  print('PSB:',tn_loc/total_loc)
  print('PNTN:',(1-(CM[4][4]/len(x_test))))
  print('Remaining Servie Time:',total_loc-tn_loc)
  print('PRST:',(1-(tn_loc/total_loc)))

In [17]:
Function(LogisticRegression());

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Train f1: 0.5380695443645084
test f1score :  0.8656273199703044
[[   0    0    0    0    0]
 [   3    1    0    0    2]
 [   7    1    1    8   29]
 [   1    1    1    1    3]
 [  56   15   28   26 1163]]
Risk Factor for High Priority nan
Risk Factor for Critical 0.09999999999999999
Risk Factor for Major 0.14347826086956522
Risk Factor for Non-Trivial 0.00025706940874035993
PTN: 1163 / 1347 = 0.8634001484780994
Saved Budget: 72566
PSB: 0.5980730716293177
PNTN: 0.13659985152190057
Remaining Servie Time: 48767
PRST: 0.40192692837068233


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in double_scalars


In [18]:
Function(DecisionTreeClassifier())

Train f1: 1.0
test f1score :  0.9042316258351893
[[   0    0    0    0    0]
 [   0    0    0    0    6]
 [   3    0    1    1   41]
 [   0    0    0    1    6]
 [   6    9    7   50 1216]]
Risk Factor for High Priority nan
Risk Factor for Critical 0.3
Risk Factor for Major 0.18043478260869567
Risk Factor for Non-Trivial 0.0004930156121610519
PTN: 1216 / 1347 = 0.902746844840386
Saved Budget: 87715
PSB: 0.722927810241237
PNTN: 0.09725315515961397
Remaining Servie Time: 33618
PRST: 0.277072189758763


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in double_scalars
